In [1]:
import sys
import torch

sys.path.append("..")

from data import get_task, get_dataloader

%load_ext autoreload
%autoreload 2

In [3]:
from omegaconf import DictConfig

cfg = DictConfig(
    {
        "model": {"name_or_path": "gpt2", "max_length": 512},
        "task": {
            "name": "wikipedia",
            "followup_char_limit": 500,
            "editor_token_limit": 50,
        },
        "data": {"test_split": 0.1, "val_split": 0.1, "n_examples": 1000, "train_batch_size": 2, "val_batch_size": 2},
        "seed": 42,
    }
)

ds = get_task(cfg, "wikipedia", "train")

dl = get_dataloader(
    ds,
    cfg,
    "train" 
)

DEBUG:data:Loading cached huggingface dataset from assets/data/cache/load_wikipedia_13ca3ba53028273144bd57b521cf2037 (rank=0)


Loading dataset from disk:   0%|          | 0/25 [00:00<?, ?it/s]

In [4]:
next(iter(dl))

{'title': ['Braggadocio (typeface)', 'Tōi Umi kara Kita Coo'],
 'first_sentence': ['Braggadocio is a geometrically constructed sans-serif stencil typeface designed by W.A.',
  'is a Japanese novel by Tamio Kageyama.'],
 'second_sentence': ['Woolley in 1930 for the Monotype Corporation.',
  'It won the Naoki Prize in 1988.'],
 'third_sentence': ['The design was based on Futura Black..',
  'It was adapted into an anime film named , released in 1993 by Toei Animation.'],
 'editor_input_ids': tensor([[   54,   970,  1636,   287, 15533,   329,   262,  2892,  8690, 10501,
             13,   383,  1486,   373,  1912,   319, 24002,  5330,  2619,   492,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
          50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
         [ 1026,  1839,   262, 11013, 18228, 15895,   287, 12122,    13,   632,
            373, 16573,   

In [ ]:
ds

Dataset({
    features: ['title', 'first_sentence', 'second_sentence', 'third_sentence', 'editor_input_ids', 'editor_attention_mask', 'target_input_ids', 'target_attention_mask'],
    num_rows: 1000
})

In [ ]:
ds

Dataset({
    features: ['title', 'first_sentence', 'second_sentence', 'third_sentence', 'editor_input_ids', 'editor_attention_mask', 'target_input_ids', 'target_attention_mask'],
    num_rows: 1000
})

In [ ]:
ds[0]

{'title': 'National Basketball League (Czech Republic)',
 'first_sentence': 'The Czech Republic National Basketball League (NBL) (), known as the Kooperativa NBL for sponsorship reasons, and previously called the Mattoni NBL for sponsorship reasons, is the top-tier level professional basketball league in the Czech Republic.',
 'second_sentence': 'It is run by the Asociace Ligových Klubů.',
 'third_sentence': 'The league operates under a promotion and relegation system.',
 'editor_input_ids': tensor([ 1026,   318,  1057,   416,   262,  1081,  1733,   558,   406,   328,
           709,   127,   121,   354, 14770,   549,   129,   107,    13,   383,
          4652, 14051,   739,   257, 12148,   290, 47652,  1080,    13, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]),
 'editor_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
 